In [1]:
import tensorflow
import keras
import sklearn
import random
import cv2
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

from keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop, SGD
from keras.layers import Input, Dense, Flatten, Dropout, Conv2D, Conv3D, GlobalAveragePooling2D
from tensorflow.keras.layers import Activation, BatchNormalization, MaxPooling2D, Add
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.applications import DenseNet121
from keras.applications.densenet import preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

import warnings
warnings.filterwarnings("ignore")

In [2]:
# kullanacağımız fonksiyonları yazıyoruz
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [3]:
# google drive'ımızı collab'a bağlıyoruz
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/
Mounted at /content/gdrive


In [5]:
# Colab'ın ana dizinine verilerimizi ekliyoruz, bunu yapmak için önce google drive'ımızın içine atmamız gerekliydi ve attık
!cp /content/gdrive/MyDrive/split_garbage_Dataset ../

cp: -r not specified; omitting directory '/content/gdrive/MyDrive/split_garbage_Dataset'


In [ ]:
# dataset'i ve içindekileri zipten çıkarttık, artık içindekilerin hepsi /darknet/data/ klasöründe
!unzip /content/gdrive/MyDrive/split_garbage_Dataset/archive.zip -d /content/gdrive/MyDrive

Streaming output truncated to the last 5000 lines.
  inflating: /content/gdrive/MyDrive/split-garbage-dataset/test/cardboard/cardboard397.jpg  
  inflating: /content/gdrive/MyDrive/split-garbage-dataset/test/cardboard/cardboard40.jpg  
  inflating: /content/gdrive/MyDrive/split-garbage-dataset/test/cardboard/cardboard401.jpg  
  inflating: /content/gdrive/MyDrive/split-garbage-dataset/test/cardboard/cardboard44.jpg  
  inflating: /content/gdrive/MyDrive/split-garbage-dataset/test/cardboard/cardboard51.jpg  
  inflating: /content/gdrive/MyDrive/split-garbage-dataset/test/cardboard/cardboard57.jpg  
  inflating: /content/gdrive/MyDrive/split-garbage-dataset/test/cardboard/cardboard63.jpg  
  inflating: /content/gdrive/MyDrive/split-garbage-dataset/test/cardboard/cardboard69.jpg  
  inflating: /content/gdrive/MyDrive/split-garbage-dataset/test/cardboard/cardboard71.jpg  
  inflating: /content/gdrive/MyDrive/split-garbage-dataset/test/cardboard/cardboard73.jpg  
  inflating: /content/gdriv

In [ ]:
path = "../input/split-garbage-dataset"

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1/255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest')

validation_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
img_shape = (128,128,3)

train_batch_size = 256
validation_batch_size = 32

train_generator = train_datagen.flow_from_directory(
    path+'/train',
    target_size = (img_shape[0],img_shape[1]),
    batch_size = train_batch_size,
    class_mode = 'categorical')

validation_generator = validation_datagen.flow_from_directory(
    path+'/valid',
    target_size = (img_shape[0],img_shape[1]),
    batch_size = validation_batch_size,
    class_mode = 'categorical',
    shuffle = False)

test_generator = test_datagen.flow_from_directory(
    path+'/test',
    target_size = (img_shape[0],img_shape[1]),
    batch_size = validation_batch_size,
    class_mode = 'categorical',
    shuffle = False)

In [ ]:
model_d = DenseNet121(weights='imagenet', include_top = False, input_shape=(128,128,3))
x = model_d.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
preds = Dense(8, activation='softmax')(x)

In [ ]:
model = Model(inputs=model_d.input, outputs=preds)
model.summary()

In [ ]:
for layer in model.layers[:-8]:
    layer.trainable = False
for layer in model.layers[-8:]:
    layer.trainable = True

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
es = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=5, verbose=1, min_lr=1e-3)
mc = ModelCheckpoint('../input/vgg16garbagedataset/VGG16 Garbage Classifier.h5', verbose=1, save_best_only=True)

datagen = ImageDataGenerator(zoom_range=0.2,horizontal_flip=True,shear_range=0.2)

datagen.fit(train_generator)

history = model.fit_generator(train_generator,
                   steps_per_epoch =  train_generator.samples/train_generator.batch_size,
                   epochs = 50,
                   validation_data = validation_generator,
                   validation_steps = validation_generator.samples/validation_generator.batch_size,
                   verbose = 0,
                   callbacks = [es,mc])

In [ ]:
score = model.evaluate(validation_generator, batch_size=32)
print('Score Accuracy : {:.2f}%'.format(score[1]*100))

In [ ]:
train_acc = history.history['acc']
val_Acc = history.history['val_acc']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1,len(train_acc)+1)

plt.plot(epochs, train_acc, 'b*-', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
         
plt.figure()

plt.plot(epochs, train_loss, 'b*-', label = 'Training Loss') # b*- mavi renkle ilgili bişey
plt.plot(epochs, val_loss, 'r', label = 'Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()

In [ ]:
ypred = model.predict(validation_generator, batch_size=32, verbose=True)

Ypred = np.argmax(Ypred, axis=1)

In [ ]:
from sklearn.preprocessing import LabelBinarizer
LABELS = ['cardboard','glass','metal','paper','plastic','trash']

labels = np.array(LABELS) 
lb = LabelBinarizer()
labels = lb.fit_transform(labels)  

In [ ]:
# imutils kütüphanesi kaggle içinde bulunmadığı için biz indiriyoruz
!pip install imutils

In [ ]:
# test için gereken kütüphaneleri import ediyoruz
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import load_model
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os

In [ ]:
def predict(imagePath):  # test edeceğimiz görseli aldığımız ve doğruluk oranını ekrana yazdırdığımız fonksiyonu yazıyoruz
    image = cv2.imread(imagePath)   # görseli aldık
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  #  görsele renk uzayı dönüşümü uyguladık
    output = image.copy()  # ana görseli kopyaladık ve 
    image = cv2.resize(image, (224,224)) # görseli VGG16'nın çalıştığı boyutlara resize ettik
    image = image.astype("float")/255.0  #  görseli yine VGG16'nın çalıştığı renklere böldük
    image = img_to_array(image)  #  görsel matrislerden oluşur, bu matris içindeki değerleri diziye aktardık
    image = np.expand_dims(image,axis=0) # görseli np.argmax()'ta olduğu gibi eksene dizer
    proba = model.predict(image)[0]  #  modelin doğruluk oranını öğrendiğimiz fonksiyonun ilk elemanının yani validation_generator'u çağıyıroruz
    idx = np.argmax(proba) # proba dizisi içindeki doğruluk oranı en yüksek olan elemanı alıyoruz
    label = lb.classes_[idx] #en yüksek doğruluk oranlarını da binarize ediyoruz
    label = "{}: {:.2f}%".format(label,proba[idx]*100)  # label değerini label'ın ismi ve doğruluk oranının 100 ile çarpılması ile formatlıyoruz
    cv2.putText(output, label, (10,25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2) # ekrana yazdırıyoruz
    
    fig = plt.figure(figsize=(10,10)) # figsize=(a,b) a ve b görselin inch birimindeki shape'idir.
    plt.grid(b=None) # görsel üzerinde matplotlib.pyplot yüzünden oluşacak olan ızgarayı siliyoruz
    plt.axis('off')  # yine matlplotlib.pyplot yüzünden oluşacak olan x ve y ekseni çizgilerini siliyoruz
    
    return plt.imshow(output)

In [ ]:
imagePath = '../input/split-garbage-dataset/test/cardboard/cardboard162.jpg'

In [ ]:
imagePath = '../input/split-garbage-dataset/test/plastic/plastic126.jpg'

In [ ]:
imagePath = '../input/split-garbage-dataset/test/metal/metal209.jpg'

In [ ]:
imagePath = '../input/split-garbage-dataset/test/trash/trash109.jpg'

In [ ]:
imagePath = '../input/split-garbage-dataset/test/paper/paper230.jpg'

In [ ]:
imagePath = '../input/split-garbage-dataset/test/glass/glass159.jpg'